In [1]:
import sklearn
import nltk
import pandas
import xlrd
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from nltk.util import ngrams
import matplotlib.pyplot as plt
from os import path
from PIL import Image
import numpy as np
from random import sample
import random
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import itertools
import xlwt
from xlwt import Workbook
import xlsxwriter 
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from collections import defaultdict
from collections import Counter

from textblob import TextBlob, Word
import math
from sklearn.feature_extraction import DictVectorizer
from collections import Counter
from sklearn.datasets import make_classification
import imblearn
from imblearn.under_sampling import RandomUnderSampler

In [2]:
def processed_abstract(abstract):   
# This is a comment: the 1st abstract value is now stored in the variable abstract
#- process it and store the tokens in some variable
    NUM_result1 = re.sub(r'\d+','NUMERIC', abstract)
    PUNC_result1 = re.sub(r'[^A-Za-z0-9-/\s]', ' ', NUM_result1)
    PMID_token1 = nltk.word_tokenize(PUNC_result1)
    
    #PMID_token1 = nltk.wordpunct_tokenize(PUNC_result1)
       
    stop_words = set(stopwords.words('english'))

    filtered_sentence1 = [w for w in PMID_token1 if not w.lower() in stop_words]
    filtered_sentence1 = []
    
    for w in PMID_token1:
        if w not in stop_words:
            if w[-1] in [".",",",":", ";"]:
                w = "".join(w[0:-1])
            filtered_sentence1.append(w)
            
    lemmatizer1 = WordNetLemmatizer()
    lemmatized_abstract1 = [lemmatizer1.lemmatize(word) for word in filtered_sentence1]
    listToStr = ' '.join([str(elem) for elem in lemmatized_abstract1])
    

    return listToStr

In [3]:
#About Set, open and read about set
full_set2 = pd.read_excel(r"C:\Users\jolsh\Documents\REU\BP_nonBP_PMID (with rest categories).xlsx", 
                          sheet_name = "BP Set")
print(full_set2)

      docID_Bibliome                                            AUTHORS  \
0                 10                      raju, ts; ray, mk; stanley, p   
1                 11                    potvin, b; raju, ts; stanley, p   
2                 14  hooker, ad; goldman, mh; markham, nh; james, d...   
3                 19  eder, jp; chan, vtw; ng, sw; rizvi, na; zachar...   
4                 39                           seewoster, t; lehmann, j   
...              ...                                                ...   
1045           10275  ho, steven c. l.; mariati; yeo, jessna h. m.; ...   
1046           10276  haryadi, ryan; ho, steven; kok, yee jiun; pu, ...   
1047           10277            wilkens, camila a.; gerdtzen, ziomara p   
1048           10278  liu, jintao; wang, jie; fan, li; chen, xinning...   
1049           10279  liu, z; dai, s; bones, j; ray, s; cha, s; karg...   

                                                  TITLE  \
0     lec18, a dominant chinese hamster 

In [4]:
#replace empty abstracts
full_set2['ABSTRACT'].fillna("ABSTRACT EMPTY",inplace=True)

In [5]:
#combine title and processed abstracts
full_set2['processed_abstracts']=(full_set2['TITLE'] + ' ' + full_set2['ABSTRACT']).apply(lambda x:processed_abstract(x))
print(full_set2)

      docID_Bibliome                                            AUTHORS  \
0                 10                      raju, ts; ray, mk; stanley, p   
1                 11                    potvin, b; raju, ts; stanley, p   
2                 14  hooker, ad; goldman, mh; markham, nh; james, d...   
3                 19  eder, jp; chan, vtw; ng, sw; rizvi, na; zachar...   
4                 39                           seewoster, t; lehmann, j   
...              ...                                                ...   
1045           10275  ho, steven c. l.; mariati; yeo, jessna h. m.; ...   
1046           10276  haryadi, ryan; ho, steven; kok, yee jiun; pu, ...   
1047           10277            wilkens, camila a.; gerdtzen, ziomara p   
1048           10278  liu, jintao; wang, jie; fan, li; chen, xinning...   
1049           10279  liu, z; dai, s; bones, j; ray, s; cha, s; karg...   

                                                  TITLE  \
0     lec18, a dominant chinese hamster 

In [6]:
#Analysis of About Set
string2 = full_set2['processed_abstracts'].str.cat(sep=', ')

def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

# Lemmatize
sentence = string2
lemma = lemmatize_with_postag(sentence)

tokenize = nltk.word_tokenize(lemma)

#will show just top x most frequent unigram, bigram, and trigrams
unigram_series2 = (pd.Series(nltk.ngrams(tokenize, 1)))[0:]
bigram_series2 = (pd.Series(nltk.ngrams(tokenize, 2)))[0:]

#stores all the unigram, bigram, and trigrams for later weighting
#unigram_series2 = (nltk.ngrams(filtered_sentence, 1))
#bigram_series2 = (nltk.ngrams(filtered_sentence, 2)))

In [7]:
joined_unigram2 = []
for x in unigram_series2:
    join = ''.join(x)
    joined_unigram2.append(join)
print(len(joined_unigram2))

158666


In [8]:
joined_bigram2 = []
for x in bigram_series2:
    join = ' '.join(x)
    joined_bigram2.append(join)
print(len(joined_bigram2))

158665


In [9]:
#Create dictionary of unique unigram terms
DF3 = defaultdict(int)
words = joined_unigram2
unique_words = set(words)

oldcount = 0
print(len(set(words)))
for x in full_set2['processed_abstracts']:
    for word in unique_words:
        if len(word) >= 2 and word in x:
            DF3[word] += 1
        else:
            oldcount = DF3.get(word)
            if oldcount is None:
                DF3[word] = 0
            else:
                DF3[word] = DF3[word]
print(len(DF3))

12068
12068


In [10]:
#Create dictionary of unique bigram terms
DF4 = defaultdict(int)
words = joined_bigram2
unique_words = set(words)

oldcount = 0
print(len(set(words)))
for x in full_set2['processed_abstracts']:
    for word in unique_words:
        if len(word) >= 2 and word in x:
            DF4[word] += 1
        else:
            oldcount = DF4.get(word)
            if oldcount is None:
                DF4[word] = 0
            else:
                DF4[word] = DF4[word]
print(len(DF4))

94217
94217


In [11]:
#merge into a dataframe
dfb3 = pd.DataFrame.from_dict(DF3, orient = 'index', columns = ['count2'])
dfb3 = dfb3.reset_index()
rslt_dfb3 = dfb3.sort_values(by = 'count2', ascending = False)

In [12]:
dfb4 = pd.DataFrame.from_dict(DF4, orient = 'index', columns = ['count2'])
dfb4 = dfb4.reset_index()
rslt_dfb4 = dfb4.sort_values(by = 'count2', ascending = False)

In [13]:
#can set to top values using integer in print statement
#ordered by document frequency for Background set unigrams use .iloc
print(rslt_dfb3.head(3500))
##ordered by document frequency for Background set bigraams
print(rslt_dfb4.head(1500))

              index  count2
6366             ch    1046
9325             re    1044
10958            te    1043
8665             se    1040
3856             er    1040
...             ...     ...
9981         concur      11
11830     integrity      11
8481        vesicle      11
1356   metabolomics      11
5038      threefold      11

[2500 rows x 2 columns]
                 index  count2
89715              e c     943
39853    hamster ovary     894
75957       chinese ha     889
80033  chinese hamster     888
34751              l c     805
...                ...     ...
12816         factor h      12
31385      defective p      12
79321   human antibody      12
23519           m base      12
31377    cell addition      12

[1000 rows x 2 columns]


In [14]:
#Background Set 
full_set = pd.read_excel(r"C:\Users\jolsh\Documents\REU\BP_nonBP_PMID (with rest categories).xlsx", sheet_name = "BP_NONBP for LR")
print(full_set)

      docID_Bibliome                                            AUTHORS  \
0                  1  li, zy; otevrel, t; gao, yj; cheng, hl; seed, ...   
1                  2  chang, rsl; lotti, vj; chen, tb; omalley, ss; ...   
2                  3  robineau, p; lonchampt, m; kucharczyk, n; krau...   
3                  4    wood, ic; roopra, a; harrington, c; buckley, nj   
4                  5                    zimprich, a; simon, t; hollt, v   
...              ...                                                ...   
9686           10275  ho, steven c. l.; mariati; yeo, jessna h. m.; ...   
9687           10276  haryadi, ryan; ho, steven; kok, yee jiun; pu, ...   
9688           10277            wilkens, camila a.; gerdtzen, ziomara p   
9689           10278  liu, jintao; wang, jie; fan, li; chen, xinning...   
9690           10279  liu, z; dai, s; bones, j; ray, s; cha, s; karg...   

                                                  TITLE  \
0     the xrcc4 gene encodes a novel pro

In [15]:
full_set['ABSTRACT'].fillna("ABSTRACT EMPTY",inplace=True)

In [16]:
full_set['processed_abstracts']=(full_set['TITLE'] + ' ' + full_set['ABSTRACT']).apply(lambda x:processed_abstract(x))
print(full_set)

      docID_Bibliome                                            AUTHORS  \
0                  1  li, zy; otevrel, t; gao, yj; cheng, hl; seed, ...   
1                  2  chang, rsl; lotti, vj; chen, tb; omalley, ss; ...   
2                  3  robineau, p; lonchampt, m; kucharczyk, n; krau...   
3                  4    wood, ic; roopra, a; harrington, c; buckley, nj   
4                  5                    zimprich, a; simon, t; hollt, v   
...              ...                                                ...   
9686           10275  ho, steven c. l.; mariati; yeo, jessna h. m.; ...   
9687           10276  haryadi, ryan; ho, steven; kok, yee jiun; pu, ...   
9688           10277            wilkens, camila a.; gerdtzen, ziomara p   
9689           10278  liu, jintao; wang, jie; fan, li; chen, xinning...   
9690           10279  liu, z; dai, s; bones, j; ray, s; cha, s; karg...   

                                                  TITLE  \
0     the xrcc4 gene encodes a novel pro

In [17]:
DF1 = defaultdict(int)
words = joined_unigram2
unique_words = set(words)
oldcount = 0
for x in full_set['processed_abstracts']:
    for word in unique_words:
        if len(word) >= 2 and word in x:
            DF1[word] += 1
        else:
            oldcount = DF1.get(word)
            if oldcount is None:
                DF1[word] = 0
            else:
                DF1[word] = DF1[word]
print(len(DF1))

12068


In [18]:
DF2 = defaultdict(int)
print(len(DF2))
words = joined_bigram2
unique_words = set(words)

oldcount = 0
for x in full_set['processed_abstracts']:
    for word in unique_words:
        if len(word) >= 2 and word in x:
            DF2[word] += 1
        else:
            oldcount = DF2.get(word)
            if oldcount is None:
                DF2[word] = 0
            else:
                DF2[word] = DF2[word]
print(len(DF2))

0
94217


In [98]:
dfb1 = pd.DataFrame.from_dict(DF1, orient = 'index', columns = ['count1'])
dfb1 = dfb1.reset_index()
dfb1 ['word'] = dfb1['index']
rslt_dfb1 = dfb1.sort_values(by = 'count1', ascending = False)

In [99]:
dfb2 = pd.DataFrame.from_dict(DF2, orient = 'index', columns = ['count1'])
dfb2 = dfb2.reset_index()
dfb2 ['word'] = dfb2['index']
rslt_dfb2 = dfb2.sort_values(by = 'count1', ascending = False)

In [100]:
#can set to top values using integer in print statement
#ordered by document frequency for Background set unigrams use .iloc
#dfb1['words'] = dfb1 ['index']
print(rslt_dfb1.head(6000))
##ordered by document frequency for Background set bigraams
#dfb2['words'] = dfb2 ['index']
print(rslt_dfb2.head(7000))

                   index  count1               word
9325                  re    9567                 re
10958                 te    9566                 te
3856                  er    9530                 er
3659                 ion    9529                ion
9569                  an    9523                 an
...                  ...     ...                ...
252                 pcna      11               pcna
2438          cocultured      11         cocultured
6459   s-NUMERIC-labeled      10  s-NUMERIC-labeled
6864          juxtacrine      10         juxtacrine
2200           tNUMERICn      10          tNUMERICn

[6000 rows x 3 columns]
                   index  count1               word
39853      hamster ovary    9008      hamster ovary
75957         chinese ha    8927         chinese ha
80033    chinese hamster    8926    chinese hamster
89715                e c    8243                e c
34751                l c    6574                l c
...                  ...     ...       

In [101]:
#create dataframe with combined counts from about and background set
result1 = pd.concat([dfb1,dfb3], axis=1)
print(result1)

                       index  count1                   word  \
0          core-glycosylated       2      core-glycosylated   
1                      he-ne       4                  he-ne   
2             biotherapeutic      16         biotherapeutic   
3      beta-d-galactosidases       2  beta-d-galactosidases   
4                  obviously       9              obviously   
...                      ...     ...                    ...   
12063                  shell       5                  shell   
12064                    ppp      11                    ppp   
12065           design-based       1           design-based   
12066           monosulfated       1           monosulfated   
12067          stirred-batch       1          stirred-batch   

                       index  count2  
0          core-glycosylated       1  
1                      he-ne       1  
2             biotherapeutic      13  
3      beta-d-galactosidases       2  
4                  obviously       1  
...      

In [102]:
#create dataframe with combined counts from about and background set
result2 = pd.concat([dfb2,dfb4], axis=1)
print(result2)

                              index  count1                          word  \
0                  upstream process       4              upstream process   
1               pathway furthermore       9           pathway furthermore   
2      membrane-bound transcription      11  membrane-bound transcription   
3                degradation target       2            degradation target   
4                  kinase implicate       6              kinase implicate   
...                             ...     ...                           ...   
94212                     prkra may       1                     prkra may   
94213                range multiple       0                range multiple   
94214       methionineamide NUMERIC       1       methionineamide NUMERIC   
94215               pattern secrete       2               pattern secrete   
94216                different shbg       1                different shbg   

                              index  count2  
0                  upstream p

In [103]:
#sort based off of frequency in about set
rslt1 = result1.sort_values(by = 'count2', ascending = False)
print(rslt1)

          index  count1      word     index  count2
6366         ch    9446        ch        ch    1046
9325         re    9567        re        re    1044
10958        te    9566        te        te    1043
8665         se    9467        se        se    1040
3856         er    9530        er        er    1040
...         ...     ...       ...       ...     ...
6175    satisfy       5   satisfy   satisfy       0
8063       bury       0      bury      bury       0
10930         w       0         w         w       0
11941  classify       5  classify  classify       0
308           g       0         g         g       0

[12068 rows x 5 columns]


In [104]:
rslt2 = result2.sort_values(by = 'count2', ascending = False)
print(rslt2)

                    index  count1                word               index  \
89715                 e c    8243                 e c                 e c   
39853       hamster ovary    9008       hamster ovary       hamster ovary   
75957          chinese ha    8927          chinese ha          chinese ha   
80033     chinese hamster    8926     chinese hamster     chinese hamster   
34751                 l c    6574                 l c                 l c   
...                   ...     ...                 ...                 ...   
82736          bear mono-       0          bear mono-          bear mono-   
22287  growth proliferate       0  growth proliferate  growth proliferate   
31786   indicate NUMERICr       0   indicate NUMERICr   indicate NUMERICr   
22290           focus ief       0           focus ief           focus ief   
37615      clone sequence       5      clone sequence      clone sequence   

       count2  
89715     943  
39853     894  
75957     889  
80033     8

In [105]:
#count 1 is for background set count 2 is for about set, looking at unigrams
dict1 = rslt1.head(3500)
print(dict1)

                 index  count1             word            index  count2
6366                ch    9446               ch               ch    1046
9325                re    9567               re               re    1044
10958               te    9566               te               te    1043
8665                se    9467               se               se    1040
3856                er    9530               er               er    1040
...                ...     ...              ...              ...     ...
1954   over-expression      27  over-expression  over-expression       6
1441     dexamethasone      31    dexamethasone    dexamethasone       6
3296            health     129           health           health       6
760            nucleic      56          nucleic          nucleic       6
7200       sulfoximine      13      sulfoximine      sulfoximine       6

[3500 rows x 5 columns]


In [106]:
#count 1 is for background set count 2 is for about set, looking at bigrams
dict2 = rslt2.head(1500)
print(dict2)

                      index  count1                  word  \
89715                   e c    8243                   e c   
39853         hamster ovary    9008         hamster ovary   
75957            chinese ha    8927            chinese ha   
80033       chinese hamster    8926       chinese hamster   
34751                   l c    6574                   l c   
...                     ...     ...                   ...   
89239       protein involve      79       protein involve   
4256            end culture      11           end culture   
47469        effect culture      16        effect culture   
18424  NUMERIC-NUMERIC cell      71  NUMERIC-NUMERIC cell   
62191            phase high      25            phase high   

                      index  count2  
89715                   e c     943  
39853         hamster ovary     894  
75957            chinese ha     889  
80033       chinese hamster     888  
34751                   l c     805  
...                     ...     ...  
8

In [107]:
#perform eGift scoring
Na = 1050
Nb = 9690
for index,row in dict1.iterrows():
        dict1.at[index,'scores'] = (((row['count2'])/(Na)) - ((row['count1'])/(Nb))) * np.log (Nb / (row['count1'])) 
print(dict1)

C:\Users\jolsh\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\jolsh\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


                 index  count1             word            index  count2  \
6366                ch    9446               ch               ch    1046   
9325                re    9567               re               re    1044   
10958               te    9566               te               te    1043   
8665                se    9467               se               se    1040   
3856                er    9530               er               er    1040   
...                ...     ...              ...              ...     ...   
1954   over-expression      27  over-expression  over-expression       6   
1441     dexamethasone      31    dexamethasone    dexamethasone       6   
3296            health     129           health           health       6   
760            nucleic      56          nucleic          nucleic       6   
7200       sulfoximine      13      sulfoximine      sulfoximine       6   

         scores  
6366   0.000545  
9325   0.000089  
10958  0.000079  
8665   0.000314

In [108]:
rslt_score1 = dict1.sort_values(by = 'scores', ascending = False).reset_index(drop = True)
rslt_score1.to_csv('eGiftScoringUnigramBP.tsv',sep = '\t')
print(rslt_score1)

#for use later 
rslt_score_for_fisher1 = dict1.sort_values(by = 'scores', ascending = False).reset_index(drop = True)
print(rslt_score_for_fisher1)



             index  count1          word         index  count2    scores
0     productivity     323  productivity  productivity     251  0.699675
1       production    1323    production    production     472  0.623226
2          product    1945       product       product     617  0.621291
3          culture    1802       culture       culture     543  0.557107
4            batch     256         batch         batch     180  0.526917
...            ...     ...           ...           ...     ...       ...
3495           cep    4045           cep           cep     159 -0.232392
3496       binding    2811       binding       binding     105 -0.235250
3497    activation    1960    activation    activation      57 -0.236502
3498       agonist    1488       agonist       agonist      12 -0.266307
3499      receptor    3599      receptor      receptor      87 -0.285798

[3500 rows x 6 columns]
             index  count1          word         index  count2    scores
0     productivity     323

In [109]:
Na = 1050
Nb = 9690
for index,row in dict2.iterrows():
        dict2.at[index,'scores'] = (((row['count2'])/(Na)) - ((row['count1'])/(Nb))) * np.log (Nb / (row['count1'])) 
print(dict2)

C:\Users\jolsh\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\jolsh\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


                      index  count1                  word  \
89715                   e c    8243                   e c   
39853         hamster ovary    9008         hamster ovary   
75957            chinese ha    8927            chinese ha   
80033       chinese hamster    8926       chinese hamster   
34751                   l c    6574                   l c   
...                     ...     ...                   ...   
89239       protein involve      79       protein involve   
4256            end culture      11           end culture   
47469        effect culture      16        effect culture   
18424  NUMERIC-NUMERIC cell      71  NUMERIC-NUMERIC cell   
62191            phase high      25            phase high   

                      index  count2    scores  
89715                   e c     943  0.007670  
39853         hamster ovary     894 -0.005706  
75957            chinese ha     889 -0.006118  
80033       chinese hamster     888 -0.006196  
34751                   l c

In [110]:
rslt_score2 = dict2.sort_values(by = 'scores', ascending = False).reset_index(drop = True)
rslt_score2.to_csv('eGiftScoringBigramBP.tsv',sep = '\t')
print(rslt_score2)

#for use in Fisher
rslt_score_for_fisher2 = dict2.sort_values(by = 'scores', ascending = False).reset_index(drop = True)
print(rslt_score_for_fisher2)

                    index  count1                 word                index  \
0                 cell cu     705              cell cu              cell cu   
1               l culture     717            l culture            l culture   
2            cell culture     632         cell culture         cell culture   
3     recombinant protein     387  recombinant protein  recombinant protein   
4               cell grow     484            cell grow            cell grow   
...                   ...     ...                  ...                  ...   
1495            NUMERIC n    1995            NUMERIC n            NUMERIC n   
1496  transfected chinese     719  transfected chinese  transfected chinese   
1497            NUMERIC r    3316            NUMERIC r            NUMERIC r   
1498     NUMERIC receptor    1008     NUMERIC receptor     NUMERIC receptor   
1499           receptor c    1276           receptor c           receptor c   

      count2    scores  
0        293  0.540620  
1

In [111]:
#FISHER STARTS HERE

In [112]:
rslt_score3 = rslt_score_for_fisher1

In [113]:
rslt_score3["a"] = rslt_score3["count2"]
rslt_score3["b"] = rslt_score3["count1"] - rslt_score3["count2"]
rslt_score3["c"] = 1050 - rslt_score3["count2"]
rslt_score3["d"] = 8640 - (rslt_score3["count1"] - rslt_score3["count2"])
#rslt_score3.to_csv('b.tsv',sep = '\t')

In [114]:
rslt_score3["a+b"] = rslt_score3["a"] + rslt_score3["b"]
rslt_score3["c+d"] = rslt_score3["c"] + rslt_score3["d"]
rslt_score3["a+c"] = rslt_score3["a"] + rslt_score3["c"]
rslt_score3["b+d"] = rslt_score3["b"] + rslt_score3["d"]
print(rslt_score3)
#(rslt_score3["b"]).to_csv('negative test.tsv',sep = '\t')

             index  count1          word         index  count2    scores    a  \
0     productivity     323  productivity  productivity     251  0.699675  251   
1       production    1323    production    production     472  0.623226  472   
2          product    1945       product       product     617  0.621291  617   
3          culture    1802       culture       culture     543  0.557107  543   
4            batch     256         batch         batch     180  0.526917  180   
...            ...     ...           ...           ...     ...       ...  ...   
3495           cep    4045           cep           cep     159 -0.232392  159   
3496       binding    2811       binding       binding     105 -0.235250  105   
3497    activation    1960    activation    activation      57 -0.236502   57   
3498       agonist    1488       agonist       agonist      12 -0.266307   12   
3499      receptor    3599      receptor      receptor      87 -0.285798   87   

         b     c     d   a+

In [115]:
#took factorial of each term individually to mkae it easier to run
joined_factorial1 = []
for x in rslt_score3["a"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial1.append(factorial)
print((joined_factorial1))

[..., 295232799039604140847618609643520000000, 4274883284060025564298013753389399649690343788366813724672000000000000, 295232799039604140847618609643520000000, 114628056373470835453434738414834942870388487424139673389282723476762012382449946252660360871841673476016298287096435143747350528228224302506311680000000000000000000000000, 117499720439091082394795827163851716458059626095095089986332811032878850206552392125984170354456588787206137541623641592892713800361142894297576474973649309989915800122274950466132528824767026591868029083847822165619507200000000000000000000000000000000000, 4733702182912325971198157282773458528972111665671644583063162654840567216299326200333597974632020795344694044141162288574741860330707871653991802372413420954892019572846468089404909755852192508097446724647826768577878987213960691804730882223315446309650598202756704313010742315578131345078364709758529795655446581758477730600169824143256656411069775872000000000000000000000000000000000000000000000000000, 33732

In [116]:
joined_factorial2 = []
for x in rslt_score3["b"]:
    #joinfact = ''.join(factorial)
        factorial = math.factorial(x)
        joined_factorial2.append(factorial)
print(len(joined_factorial2))

3500


In [117]:
joined_factorial3 = []
for x in rslt_score3["c"]:
    if x < 0:
        joined_factorial3.append(1)
    else:
        factorial = math.factorial(x)
        #joinfact = ''.join(factorial)
        joined_factorial3.append(factorial)
print(len(joined_factorial3))

3500


In [118]:
joined_factorial4 = []
for x in rslt_score3["d"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial4.append(factorial)
print(len(joined_factorial4))

3500


In [119]:
joined_factorial5 = []
for x in rslt_score3["a+b"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial5.append(factorial)
print(len(joined_factorial5))

3500


In [120]:
joined_factorial6 = []
for x in rslt_score3["c+d"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial6.append(factorial)
print(len(joined_factorial6))

3500


In [121]:
joined_factorial7 = []
for x in rslt_score3["a+c"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial7.append(factorial)
print(len(joined_factorial7))

3500


In [122]:
joined_factorial8 = []
for x in rslt_score3["b+d"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial8.append(factorial)
print(len(joined_factorial8))

3500


In [123]:
n = math.factorial(9690)

In [124]:
fisher_list_numerator = []
for x in range(3500):
    numerator = joined_factorial5[x] * joined_factorial6[x] * joined_factorial7[x] * joined_factorial8[x]
    fisher_list_numerator.append(numerator)
print(len(fisher_list_numerator))



3500


In [125]:
fisher_list_denominator = []

for x in range(3500):
    denominator = joined_factorial1[x] * joined_factorial2[x] * joined_factorial3[x] * joined_factorial4[x] * n 
    fisher_list_denominator.append(denominator)
print(len(fisher_list_denominator))

3500


In [126]:
fisher_list_score = []
for x in range(3500):
    value = (fisher_list_numerator[x]) / (fisher_list_denominator[x])
    fisher_list_score.append(value)
#print((fisher_list_score))

rslt_score3["Fisher Scores"] = fisher_list_score
#print(rslt_score1)
new_order = rslt_score3.sort_values(by = 'Fisher Scores', ascending = False).reset_index(drop = True)
print(new_order)

               index  count1            word           index  count2  \
0        facilitates      55     facilitates     facilitates       6   
1       hypoxanthine      55    hypoxanthine    hypoxanthine       6   
2     factor-NUMERIC      54  factor-NUMERIC  factor-NUMERIC       6   
3          differing      54       differing       differing       6   
4          microbial      54       microbial       microbial       6   
...              ...     ...             ...             ...     ...   
3495     recombinant    1846     recombinant     recombinant     512   
3496         culture    1802         culture         culture     543   
3497      production    1323      production      production     472   
3498    productivity     323    productivity    productivity     251   
3499         product    1945         product         product     617   

        scores    a     b     c     d   a+b   c+d   a+c   b+d  Fisher Scores  
0     0.000198    6    49  1044  8591    55  9635  1050 

In [127]:
final_scoring = new_order.drop(['a','b', 'c', 'd', 'a+b', 'c+d', 'a+c', 'b+d'],axis = 1)
print(final_scoring)
final_scoring.to_csv('FisherUnigramBP.tsv',sep = '\t')

               index  count1            word           index  count2  \
0        facilitates      55     facilitates     facilitates       6   
1       hypoxanthine      55    hypoxanthine    hypoxanthine       6   
2     factor-NUMERIC      54  factor-NUMERIC  factor-NUMERIC       6   
3          differing      54       differing       differing       6   
4          microbial      54       microbial       microbial       6   
...              ...     ...             ...             ...     ...   
3495     recombinant    1846     recombinant     recombinant     512   
3496         culture    1802         culture         culture     543   
3497      production    1323      production      production     472   
3498    productivity     323    productivity    productivity     251   
3499         product    1945         product         product     617   

        scores  Fisher Scores  
0     0.000198   1.706034e-01  
1     0.000198   1.706034e-01  
2     0.000735   1.704604e-01  
3     0

In [128]:
#Fisher Test Bigram

In [129]:
rslt_score4 = rslt_score_for_fisher2

In [130]:
rslt_score4["a"] = rslt_score4["count2"]
rslt_score4["b"] = rslt_score4["count1"] - rslt_score4["count2"]
rslt_score4["c"] = 1050 - rslt_score4["count2"]
rslt_score4["d"] = 8640 - rslt_score4["b"]
#rslt_score4.to_csv('b.tsv',sep = '\t')
#(rslt_score4["b"]).to_csv('negative test.tsv',sep = '\t')

In [131]:
rslt_score4["a+b"] = rslt_score4["a"] + rslt_score4["b"]
rslt_score4["c+d"] = rslt_score4["c"] + rslt_score4["d"]
rslt_score4["a+c"] = rslt_score4["a"] + rslt_score4["c"]
rslt_score4["b+d"] = rslt_score4["b"] + rslt_score4["d"]
print(rslt_score4)

                    index  count1                 word                index  \
0                 cell cu     705              cell cu              cell cu   
1               l culture     717            l culture            l culture   
2            cell culture     632         cell culture         cell culture   
3     recombinant protein     387  recombinant protein  recombinant protein   
4               cell grow     484            cell grow            cell grow   
...                   ...     ...                  ...                  ...   
1495            NUMERIC n    1995            NUMERIC n            NUMERIC n   
1496  transfected chinese     719  transfected chinese  transfected chinese   
1497            NUMERIC r    3316            NUMERIC r            NUMERIC r   
1498     NUMERIC receptor    1008     NUMERIC receptor     NUMERIC receptor   
1499           receptor c    1276           receptor c           receptor c   

      count2    scores    a     b     c     d   a+b

In [132]:
joined_factorial9 = []
for x in rslt_score4["a"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial9.append(factorial)
print(len(joined_factorial9))

1500


In [133]:
joined_factorial10 = []
for x in rslt_score4["b"]:
    #joinfact = ''.join(factorial)
        factorial = math.factorial(x)
        joined_factorial10.append(factorial)
print(len(joined_factorial10))

1500


In [134]:
joined_factorial11 = []
for x in rslt_score4["c"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial11.append(factorial)
print(len(joined_factorial11))

1500


In [135]:
joined_factorial12 = []
for x in rslt_score4["d"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial12.append(factorial)
print(len(joined_factorial12))

1500


In [136]:
joined_factorial13 = []
for x in rslt_score4["a+b"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial13.append(factorial)
print(len(joined_factorial13))

1500


In [137]:
joined_factorial14 = []
for x in rslt_score4["c+d"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial14.append(factorial)
print(len(joined_factorial14))

1500


In [138]:
joined_factorial15 = []
for x in rslt_score4["a+c"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial15.append(factorial)
print(len(joined_factorial15))

1500


In [139]:
joined_factorial16 = []
for x in rslt_score4["b+d"]:
    factorial = math.factorial(x)
    #joinfact = ''.join(factorial)
    joined_factorial16.append(factorial)
print(len(joined_factorial16))

1500


In [140]:
fisher_list_numerator2 = []
for x in range(1500):
    numerator2 = joined_factorial13[x] * joined_factorial14[x] * joined_factorial15[x] * joined_factorial16[x]
    fisher_list_numerator2.append(numerator2)
print(len(fisher_list_numerator2))

1500


In [141]:
fisher_list_denominator2 = []

for x in range(1500):
    denominator2 = joined_factorial9[x] * joined_factorial10[x] * joined_factorial11[x] * joined_factorial12[x] * n 
    fisher_list_denominator2.append(denominator2)
print(len(fisher_list_denominator2))

1500


In [142]:
fisher_list_score2 = []
for x in range(1500):
    value2 = (fisher_list_numerator2[x]) / (fisher_list_denominator2[x])
    fisher_list_score2.append(value2)
#print((fisher_list_score))

rslt_score4["Fisher Scores"] = fisher_list_score2
#print(rslt_score1)
new_order2 = rslt_score4.sort_values(by = 'Fisher Scores', ascending = False).reset_index(drop = True)
print(new_order2)

                    index  count1                 word                index  \
0                cell may      82             cell may             cell may   
1                  well r      81               well r               well r   
2          cell determine      81       cell determine       cell determine   
3         protein involve      79      protein involve      protein involve   
4          cellular level      78       cellular level       cellular level   
...                   ...     ...                  ...                  ...   
1495              batch c     177              batch c              batch c   
1496  recombinant protein     387  recombinant protein  recombinant protein   
1497         cell culture     632         cell culture         cell culture   
1498            l culture     717            l culture            l culture   
1499              cell cu     705              cell cu              cell cu   

      count2    scores    a    b     c     d  a+b  

In [143]:
final_scoring2 = new_order2.drop(['a','b', 'c', 'd', 'a+b', 'c+d', 'a+c', 'b+d'],axis = 1)
print(final_scoring2)
final_scoring2.to_csv('FisherBigramBP.tsv',sep = '\t')

                    index  count1                 word                index  \
0                cell may      82             cell may             cell may   
1                  well r      81               well r               well r   
2          cell determine      81       cell determine       cell determine   
3         protein involve      79      protein involve      protein involve   
4          cellular level      78       cellular level       cellular level   
...                   ...     ...                  ...                  ...   
1495              batch c     177              batch c              batch c   
1496  recombinant protein     387  recombinant protein  recombinant protein   
1497         cell culture     632         cell culture         cell culture   
1498            l culture     717            l culture            l culture   
1499              cell cu     705              cell cu              cell cu   

      count2    scores  Fisher Scores  
0          

In [144]:
print(final_scoring)
print(final_scoring2)

               index  count1            word           index  count2  \
0        facilitates      55     facilitates     facilitates       6   
1       hypoxanthine      55    hypoxanthine    hypoxanthine       6   
2     factor-NUMERIC      54  factor-NUMERIC  factor-NUMERIC       6   
3          differing      54       differing       differing       6   
4          microbial      54       microbial       microbial       6   
...              ...     ...             ...             ...     ...   
3495     recombinant    1846     recombinant     recombinant     512   
3496         culture    1802         culture         culture     543   
3497      production    1323      production      production     472   
3498    productivity     323    productivity    productivity     251   
3499         product    1945         product         product     617   

        scores  Fisher Scores  
0     0.000198   1.706034e-01  
1     0.000198   1.706034e-01  
2     0.000735   1.704604e-01  
3     0

In [145]:
#sort scores based off fisher test to be used as custom vocabulary
feature_list = final_scoring.append(final_scoring2, ignore_index = True)
feature_list.sort_values(by = 'Fisher Scores', ascending = False, inplace = True)
feature_list.to_csv('combinedFeatureListBPFisher.tsv',sep = '\t')
print(feature_list)

               index  count1            word           index  count2  \
0        facilitates      55     facilitates     facilitates       6   
1       hypoxanthine      55    hypoxanthine    hypoxanthine       6   
2     factor-NUMERIC      54  factor-NUMERIC  factor-NUMERIC       6   
3          differing      54       differing       differing       6   
4          microbial      54       microbial       microbial       6   
...              ...     ...             ...             ...     ...   
3495     recombinant    1846     recombinant     recombinant     512   
3496         culture    1802         culture         culture     543   
3497      production    1323      production      production     472   
3498    productivity     323    productivity    productivity     251   
3499         product    1945         product         product     617   

        scores  Fisher Scores  
0     0.000198   1.706034e-01  
1     0.000198   1.706034e-01  
2     0.000735   1.704604e-01  
3     0

In [146]:
feature_list2 = rslt_score1.append(rslt_score2, ignore_index = True)
feature_list2.sort_values(by = 'scores', ascending = False, inplace = True)
feature_list2.to_csv('combinedFeatureListBPeGift.tsv',sep = '\t')
print(feature_list2)

             index  count1          word         index  count2    scores
0     productivity     323  productivity  productivity     251  0.699675
1       production    1323    production    production     472  0.623226
2          product    1945       product       product     617  0.621291
3          culture    1802       culture       culture     543  0.557107
3500       cell cu     705       cell cu       cell cu     293  0.540620
...            ...     ...           ...           ...     ...       ...
3495           cep    4045           cep           cep     159 -0.232392
3496       binding    2811       binding       binding     105 -0.235250
3497    activation    1960    activation    activation      57 -0.236502
3498       agonist    1488       agonist       agonist      12 -0.266307
3499      receptor    3599      receptor      receptor      87 -0.285798

[5000 rows x 6 columns]


In [162]:
#pick the vocab set and length to be used
myvocab ={}
count = 0
top = feature_list['word'].head(2700)
top.to_csv('vocabBP.tsv',sep = '\t')
for x in top:
    myvocab[str(x)] = count
    count+=1
#print(myvocab)

In [163]:
#correct distribution with undersampling
undersample = full_set
undersample.sort_values(by = 'Label', ascending = False, inplace = True)
#print(undersample)
y = undersample[:1000]
#print(y)
x = undersample[1000:]
#print(len(x))
#print(x)
z = x.sample(frac=0.1, replace=True, random_state=1)
#print(z)

frames = [y, z]
new_set = pd.concat(frames)
#print(new_set)
new_set.to_csv('undersample.tsv',sep = '\t')
new_set.sort_values(by = 'docID_Bibliome', ascending = False, inplace = True)
#print(new_set)

In [164]:
training_data, testing_data = train_test_split(new_set,random_state = 2000, test_size = 0.2)
print(len(training_data))
print(len(testing_data))

1495
374


In [165]:
Y_train = training_data['Label'].values
Y_test = testing_data['Label'].values

In [166]:
def extract_features(df,field,training_data,testing_data,type="binary"):
    if "binary" in type:
        
        # BINARY FEATURE REPRESENTATION
        cv= CountVectorizer(binary=True, max_df=0.9)#my list of top terms
        cv.fit_transform(training_data[field].values)#transform my datafframe field using the list of top terms
        
        train_feature_set=cv.transform(training_data[field].values)
        test_feature_set=cv.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,cv
  
    elif "counts" in type:
        
        # COUNT BASED FEATURE REPRESENTATION
        cv= CountVectorizer(binary=False, max_df=0.95)
        cv.fit_transform(training_data[field].values)
        
        train_feature_set=cv.transform(training_data[field].values)
        test_feature_set=cv.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,cv
    
    
    elif "tf-idf" in type:    
        
        # TF-IDF BASED FEATURE REPRESENTATION
        tfidf_vectorizer=TfidfVectorizer(use_idf=True, min_df = 0.1 , max_df=0.95, max_features = 235, ngram_range = (1,3))
        #print(tfidf_vectorizer)
        tfidf_vectorizer.fit_transform(training_data[field].values)
        
        names = tfidf_vectorizer.get_feature_names()
        #print(names)
        
        feature_set = tfidf_vectorizer.transform(df[field].values)
        
        train_feature_set=tfidf_vectorizer.transform(training_data[field].values)
        test_feature_set=tfidf_vectorizer.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,tfidf_vectorizer, feature_set
    
    else:
        #method for Fisher terms
        vectorizer = TfidfVectorizer(use_idf = False,max_df = 0.95, vocabulary=myvocab)
        vectorizer.fit_transform(training_data[field].values)
        #print(X)
        
        feature_set = vectorizer.transform(df[field].values)
        
        train_feature_set= vectorizer.transform(training_data[field].values)
        test_feature_set= vectorizer.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set, vectorizer, feature_set

In [167]:
def get_top_k_predictions(model,X_test,k):
    
# get probabilities instead of predicted labels, since we want to collect top 3
    probs = model.predict_proba(X_test)

# GET TOP K PREDICTIONS BY PROB - note these are just index
    best_n = np.argsort(probs, axis=1)[:,-k:]
    
# GET CATEGORY OF PREDICTIONS
    preds=[[model.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]
    
# REVERSE CATEGORIES - DESCENDING ORDER OF IMPORTANCE
    preds=[ item[::-1] for item in preds]
    
    return preds

In [168]:
field = 'processed_abstracts'
feature_rep = 'other'
top_k = 1

In [169]:
X_train,X_test,feature_transformer, X = extract_features(new_set,field,training_data,testing_data,type=feature_rep)


In [170]:
print(feature_transformer)

TfidfVectorizer(max_df=0.95, use_idf=False,
                vocabulary={'-/': 508, '-NUMERIC NUMERIC': 1877,
                            '-NUMERIC c': 2661, '-NUMERIC cho': 482,
                            '-NUMERIC express': 512, '-NUMERIC increase': 790,
                            '-NUMERIC k': 829, '-NUMERIC mg': 2206,
                            '-NUMERIC result': 290, '-NUMERIC show': 252,
                            '-NUMERICa': 357, '-NUMERICb': 77,
                            '-NUMERICh': 2291, '-anchored': 739, '-beta': 664,
                            '-bound': 2410, '-cNUMERIC': 48, '-cells': 613,
                            '-cho': 1165, '-cho cell': 326, '-containing': 886,
                            '-cys': 160, '-defective': 832, '-deficient': 1227,
                            '-deficient cho': 2596, '-depleted': 96,
                            '-derived': 894, '-derived cell': 2151,
                            '-directed': 1984, '-encoding': 1350, ...})


In [171]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',multi_class = 'auto', random_state=10, C=1,max_iter=1000)
model=scikit_log_reg.fit(X_train,Y_train)

[LibLinear]

In [172]:
preds=get_top_k_predictions(model,X_test,top_k)
Y_predicted = ([x[0] for x in preds])
print(Y_predicted)

[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 

In [173]:
cm = confusion_matrix(Y_test,Y_predicted)
print(cm)

[[ 93  83]
 [ 43 155]]


In [174]:
print(sklearn.metrics.classification_report(Y_test,Y_predicted))

              precision    recall  f1-score   support

           0       0.68      0.53      0.60       176
           1       0.65      0.78      0.71       198

    accuracy                           0.66       374
   macro avg       0.67      0.66      0.65       374
weighted avg       0.67      0.66      0.66       374



In [175]:
Y = new_set['Label']

scores = cross_val_score(model, X, Y, cv=10)
print(scores)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][0.73262032 0.71657754 0.70053476 0.70588235 0.72192513 0.67379679
 0.6684492  0.60962567 0.63101604 0.60752688]


In [176]:
avg = scores.mean()
print("Accuracy of the model with a 10-fold cross validation: ", avg * 100)

Accuracy of the model with a 10-fold cross validation:  67.6795468920706
